In [18]:
import numpy as np
import yaml
import json
import matplotlib.pyplot as plt
import pandas as pd

from matplotlib import gridspec
from caesar.logger.logger import setup_logger
from pathlib import Path
from typing import List, Dict, Any
from scipy.stats import gaussian_kde
from typing import List, Tuple
import numpy.typing as npt
from matplotlib import rcParams

config = {
    "font.family":'serif',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

In [19]:
file_path="./pm3m/lmps-10000.dat"

In [20]:
with open(file_path,'r') as f:
    data_tmp=f.readlines()
data_tmp=[i.strip().split() for i in data_tmp]

In [21]:
data_df = pd.DataFrame(data_tmp, columns=[
                       'Step', 'Temp', 'PotEng', 'KinEng', 'TotEng', 'Press', 'Volume', 'Lx', 'Ly', 'Lz', 'Xy', 'Xz', 'Yz'])
data_df

,Step,Temp,PotEng,KinEng,TotEng,Press,Volume,Lx,Ly,Lz,Xy,Xz,Yz
0,0,0,-39752.398,0,-39752.398,46795.163,59900.45,39.127013,39.127013,39.127013,0,0,0
1,100,0.00015210344,-39758.716,9.8284905e-05,-39758.716,8246.6309,61101.541,39.386788,39.386804,39.386814,3.5326059e-08,7.2431938e-07,4.1255747e-06
2,200,0.00023564289,-39762.871,0.00015226571,-39762.871,9795.9918,61048.139,39.375332,39.375327,39.375314,5.7626876e-06,-3.5634261e-06,3.4746638e-07
3,300,0.0010425219,-39763.469,0.00067364788,-39763.468,9929.6631,61044.882,39.374639,39.37462,39.374613,-9.2307326e-06,3.4912576e-06,-1.5874335e-05
4,400,0.0018143885,-39764.412,0.0011724061,-39764.411,10001.283,61044.851,39.374594,39.374634,39.374624,-1.2841121e-05,2.5334714e-05,2.2354064e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20015,1999600,497.44742,-39452.629,321.43633,-39131.193,8856.337,62506.931,39.684102,39.71006,39.66533,-0.018734112,0.011625946,-0.031051127
20016,1999700,494.96636,-39449.917,319.83314,-39130.084,9467.558,62482.378,39.666461,39.661913,39.715536,0.032367626,0.0019631533,0.053154093
20017,1999800,509.41866,-39455.018,329.1718,-39125.846,9291.6298,62486.974,39.700602,39.677486,39.668726,-0.059965272,0.013196611,-0.055095819
20018,1999900,503.91503,-39440.183,325.61551,-39114.567,10238.154,62527.983,39.692665,39.671032,39.709157,0.07381169,0.018735545,0.057210181


In [22]:
temp_lattice=data_df[['Temp','Lx','Ly','Lz']].astype(float) # 提取温度和晶格常数
temp_lattice

,Temp,Lx,Ly,Lz
0,0.000000,39.127013,39.127013,39.127013
1,0.000152,39.386788,39.386804,39.386814
2,0.000236,39.375332,39.375327,39.375314
3,0.001043,39.374639,39.374620,39.374613
4,0.001814,39.374594,39.374634,39.374624
...,...,...,...,...
20015,497.447420,39.684102,39.710060,39.665330
20016,494.966360,39.666461,39.661913,39.715536
20017,509.418660,39.700602,39.677486,39.668726
20018,503.915030,39.692665,39.671032,39.709157


In [41]:
lattice_arr = []
l_x = []
l_y = []
l_z = []
for i in range(0, 300+1, 25):
    filtered_data = temp_lattice[(
        temp_lattice['Temp'] > i-5) & (temp_lattice['Temp'] < i+5)]  # 过滤温度
    lattice_avg = list(filtered_data[['Lx', 'Ly', 'Lz']].mean())  # 计算平均晶格常数
    l_x.append(lattice_avg[0])
    l_y.append(lattice_avg[1])
    l_z.append(lattice_avg[2])
lattice_avg = [l_x, l_y, l_z]
lattice_avg

[[39.36737010000001,
  39.46586559236326,
  39.47658165562249,
  39.48363039117352,
  39.485693785571144,
  39.45399472772773,
  39.47135708796764,
  39.48656505888325,
  39.50146539979444,
  39.51644683649789,
  39.53139485556781,
  39.546831209276014,
  39.56107302790697],
 [39.36899176666667,
  39.35459831785346,
  39.369950022088354,
  39.39127015245737,
  39.413323161322644,
  39.459073356356356,
  39.47146600101113,
  39.48594025279188,
  39.50194598561151,
  39.51743126054853,
  39.53177721940463,
  39.54686192533937,
  39.5611205244186],
 [39.36744913333334,
  39.35121285036119,
  39.37155411345382,
  39.389989620862586,
  39.41435499398798,
  39.449141968968966,
  39.468184702730035,
  39.485778332994926,
  39.50132687975334,
  39.51547748945148,
  39.5310439106946,
  39.545819459276025,
  39.5604208116279]]

In [36]:
filtered_data=temp_lattice[(temp_lattice['Temp']>0-5) & (temp_lattice['Temp']<0+5)] # 过滤温度异常值
lattice_avg=filtered_data[['Lx','Ly','Lz']].mean() # 计算平均晶格常数
list(lattice_avg)

[39.36737010000001, 39.36899176666667, 39.36744913333334]